In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import os
import wget
from sklearn.metrics import recall_score
from utils import *

## Overview
We have a database (ULL_database) with information about primary and secondary education students in the Canary Islands 
for 4 academic years. There is information about their academic performance and 
contextual information (about their families, teachers, and school). The database contains a subset of data 
in the form of panel data, meaning information about the same students at different points in time (ULL_panel_data).

Machine learning algorithms can be used to predict at-risk students. 
A student is considered at risk if they are anticipated to have low academic performance in the future. 
Detecting these students would allow for corrective measures to be taken in advance.

As a measure of academic performance, we have the variables "scores".
We have academic performance in Mathematics and in Spanish Language

We specify a model to predict at-risk students. Utilizing the panel data,
the model aims to forecast whether the student will be at risk in the future (in 6th grade)
based on various predictors of current academic performance (3rd grade).

Each observation (row) in ULL_panel_data is a student, with their academic performance in sixth grade 
and their predictors of academic performance from third grade (columns).

## Load and preprocess data

In [2]:
DATA = 'data/'
data = pd.read_csv(os.path.join(DATA, 'ULL_panel_data.csv'), sep=';')

In [3]:
data

,id_grade,id_student_16_19,score_MAT,level_MAT,score_LEN,level_LEN,id_student,id_student_original,id_year,id_class_group,...,p331a,p331b,p331c,p331d,p331e,p331f,p331g,p331j,pfc,rep
0,6,1,474.9944,2,385.1411,1,20342,1431,2016,A,...,4.0,4.0,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN
1,6,2,508.8362,3,469.4856,2,2819,1432,2016,A,...,4.0,NaN,4.0,4.0,3.0,4.0,NaN,NaN,NaN,NaN
2,6,3,590.2816,3,591.1398,3,19276,1436,2016,A,...,4.0,4.0,4.0,4.0,2.0,4.0,NaN,NaN,NaN,NaN
3,6,5,394.4247,1,493.7984,2,14078,1439,2016,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,6,530.0070,3,500.0860,3,1695,1447,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15969,6,17267,599.0310,3,615.6177,4,12265,40236,2016,A,...,4.0,NaN,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN
15970,6,17268,538.5835,3,647.9100,4,15982,40238,2016,A,...,4.0,NaN,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN
15971,6,17269,537.8327,3,445.1313,2,9965,40240,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15972,6,17270,468.8731,2,546.8035,3,1137,40246,2016,B,...,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN


In [4]:
# Select only the data we want to work for
data = data[['id_student_16_19', 'score_MAT', 'score_LEN', 'score_MAT3', 'score_LEN3', 'a1',
             'mother_education', 'father_education', 'mother_occupation', 'father_occupation', 
             'inmigrant_second_gen', 'start_schooling_age', 'books', 'f12a', 'public_private', 
             'capital_island', 'd14', 'ESCS', 'id_school']]

In [5]:
# Drop observations with missing data in any of the variables that we will use in the models
# Here, synthetic data methods can be used instead to fill in missing values

missing_columns = ['score_MAT3', 'a1', 'mother_education', 'father_education',
    'mother_occupation', 'father_occupation', 'inmigrant_second_gen',
    'start_schooling_age', 'books', 'f12a', 'public_private',
    'capital_island', 'd14']

data = data.dropna(subset=missing_columns)

In [6]:
data

,id_student_16_19,score_MAT,score_LEN,score_MAT3,score_LEN3,a1,mother_education,father_education,mother_occupation,father_occupation,inmigrant_second_gen,start_schooling_age,books,f12a,public_private,capital_island,d14,ESCS,id_school
4,6,530.0070,500.0860,368.65,339.47,1,3.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,2,2,4.0,0.132756,2443
5,7,531.9280,459.4065,387.36,566.44,1,4.0,4.0,4.0,3.0,1.0,1.0,4.0,5.0,2,1,1.0,1.069410,1368
7,9,578.3741,630.4484,549.89,635.53,1,2.0,2.0,3.0,2.0,1.0,2.0,2.0,2.0,2,1,1.0,-1.166950,2500
8,10,481.1748,497.1981,592.00,668.26,2,4.0,4.0,4.0,4.0,1.0,1.0,3.0,4.0,1,1,1.0,0.976453,1610
11,13,521.5593,655.0537,490.28,524.98,2,4.0,4.0,3.0,3.0,1.0,1.0,2.0,5.0,2,1,1.0,-0.134441,1859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15968,17266,522.6458,611.0034,574.83,591.86,1,4.0,3.0,3.0,3.0,1.0,1.0,4.0,2.0,2,1,2.0,0.759928,2413
15969,17267,599.0310,615.6177,629.84,460.75,1,4.0,3.0,4.0,3.0,1.0,1.0,4.0,2.0,2,1,2.0,1.410322,2203
15970,17268,538.5835,647.9100,600.20,542.78,1,4.0,2.0,3.0,3.0,1.0,1.0,2.0,3.0,2,1,2.0,0.035227,2095
15971,17269,537.8327,445.1313,610.26,600.15,2,4.0,1.0,3.0,3.0,1.0,1.0,3.0,2.0,2,1,2.0,0.382896,2097


In [7]:
cols = data.columns 
data = pd.DataFrame(data.values.flatten().reshape(-1, data.shape[1]), columns=cols)

In [8]:
# Generate quartiles of scores in sixth grade
data['scores_MATq'] = pd.qcut(data['score_MAT'], 4, labels=["1", "2", "3","4"])
data['scores_MATq'] = data['scores_MATq'].astype(int)
data['scores_LENq'] = pd.qcut(data['score_LEN'], 4, labels=["1", "2", "3","4"])
data['scores_LENq'] = data['scores_LENq'].astype(int)

In [9]:
# Generate median and percentiles 25 and 75 of socioeconomic status (ESCS)
median_ESCS = data['ESCS'].median()
p25_ESCS = data['ESCS'].quantile(0.25)
p75_ESCS = data['ESCS'].quantile(0.75)

# Initialize with null values
data['ESCS_median'] = pd.Series([np.nan] * len(data))
data.loc[data['ESCS'] >= median_ESCS, 'ESCS_median'] = 2
data.loc[data['ESCS'] < median_ESCS, 'ESCS_median'] = 1
data.loc[data['ESCS_median'] == 0, 'ESCS_median'] = np.nan

# Initialize with null values
data['ESCS_p25_p75'] = pd.Series([np.nan] * len(data))
data.loc[data['ESCS'] >= p75_ESCS, 'ESCS_p25_p75'] = 2
data.loc[data['ESCS'] < p25_ESCS, 'ESCS_p25_p75'] = 1
data.loc[(data['ESCS'] >= p25_ESCS) & (data['ESCS'] < p75_ESCS), 'ESCS_p25_p75'] = np.nan

# Some data corrections to make the final results
# Variable d14 top category(4) is the "bad" category (more than 50% of teachers change school), so the results must be inverted
data['d14'] = data['d14'].apply(lambda x: 1 if x == 1 else 0)

## Models

The goal of the model is to predict the academic performance in sixth grade ($Y_t$)
using information from the same student in third grade, specifically:

1.  Academic performance in third grade ($Y_{t-1}$)

2.  Sensitive factors or circumstances ($C$)

3.  Predictors uncorrelated with circumstances, also called "effort" ($X$)

**Model 1**:    $$Y_t = α + β1Y_{t-1} + ε$$

**Model 2**:    $$Y_t = α + β1Y_{t-1} + β2C + ε$$

**Model 3**:    

> First step: $$Y_{t-1} = α + β2C + ν$$

- Recover the prediction of $Y_{t-1}$ (academic performance due to circumstances, $C$): $\hat{Y}_{t-1}$

- Recover the residual $ν$ (academic performance due to effort, $X$): $\hat{ν}$

> Second step: $$Y_t = α + β1\hat{Y}_{t-1} + β2\hat{ν} + ε$$

- Recover the prediction of $Y_t$ only due to $\hat{Y}_{t-1}$ (only due to circumstances)

- Recover the prediction of $Y_t$ only due to $\hat{ν}$ (only due to effort)

In theory...

**Model 1**: Using only the academic performance in third grade (benchmark)

**Model 2**: Using the academic performance + circumstances in third grade (less fair - more socially desirable)

**Model 3**: Using the circumstances + effort in third grade (close to Model 2)

- Prediction exclusively of circumstances of Model 3 (much less fair - much more socially desirable)
    
- Prediction exclusively of effort of Model 3 (much more fair - much less socially desirable)

Let's prove it

In [10]:
# Variables for the models
Y_t_1 = "score_MAT3"
c = data[["a1", "mother_education", "father_education", "mother_occupation", "father_occupation", 
      "inmigrant_second_gen", "start_schooling_age", "books", "f12a", "public_private", "capital_island", "d14"]]
circumstances = ["a1", "mother_education", "father_education", "mother_occupation", "father_occupation", 
      "inmigrant_second_gen", "start_schooling_age", "books", "f12a", "public_private", "capital_island", "d14"]

# Dummy variables (all variables C are categorical variables)
dummy_variables = pd.get_dummies(c, columns=circumstances, drop_first = True)

# Join Y_t_1 + C
data_combined = pd.concat([data[Y_t_1], dummy_variables], axis=1)

In [11]:
# Model 1
model1 = sm.OLS(data["score_MAT"], sm.add_constant(data[Y_t_1])).fit()
print(model1.summary())
data['model1_pred'] = model1.fittedvalues

                            OLS Regression Results                            
Dep. Variable:              score_MAT   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.253
Method:                 Least Squares   F-statistic:                     2810.
Date:                Thu, 01 Aug 2024   Prob (F-statistic):               0.00
Time:                        16:40:06   Log-Likelihood:                -48733.
No. Observations:                8290   AIC:                         9.747e+04
Df Residuals:                    8288   BIC:                         9.748e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        232.0412      5.403     42.946      0.0

In [12]:
# Model 2
model2 = sm.OLS(data["score_MAT"], sm.add_constant(data_combined.astype(np.float64))).fit()
print(model2.summary())
data['model2_pred'] = model2.fittedvalues

                            OLS Regression Results                            
Dep. Variable:              score_MAT   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     112.9
Date:                Thu, 01 Aug 2024   Prob (F-statistic):               0.00
Time:                        16:40:06   Log-Likelihood:                -48641.
No. Observations:                8290   AIC:                         9.734e+04
Df Residuals:                    8262   BIC:                         9.754e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [13]:
# Model 3
model3 = sm.OLS(data["score_MAT3"], sm.add_constant(dummy_variables.astype(np.float64))).fit()
print(model3.summary())

# First step
data['Y_t_1_hat'] = model3.fittedvalues
data['ν_hat'] = model3.resid

# Second step
model4 = sm.OLS(data["score_MAT"], sm.add_constant(data[["Y_t_1_hat", "ν_hat"]])).fit()
print(model4.summary())
data['model3_pred'] = model4.fittedvalues

# Prediction exclusively of circumstances
data['model3_pred_circum'] = model4.params['const'] + model4.params['Y_t_1_hat'] * data['Y_t_1_hat']
# Prediction exclusively of effort
mean_circu = data['Y_t_1_hat'].mean()
data['mean_circu'] = mean_circu
data['model3_pred_effort'] = (model4.params['const'] + 
                          model4.params['ν_hat'] * data['ν_hat'] + 
                          model4.params['Y_t_1_hat'] * mean_circu)

                            OLS Regression Results                            
Dep. Variable:             score_MAT3   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     32.51
Date:                Thu, 01 Aug 2024   Prob (F-statistic):          3.71e-153
Time:                        16:40:06   Log-Likelihood:                -48947.
No. Observations:                8290   AIC:                         9.795e+04
Df Residuals:                    8263   BIC:                         9.814e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [14]:
# Transform predictions(continuous) to quartiles(categorical)

data['scores_MAT_pred1'] = pd.qcut(data['model1_pred'], 4, labels=["1", "2", "3","4"])
data['scores_MAT_pred1'] = data['scores_MAT_pred1'].astype(int)
data['scores_MAT_pred2'] = pd.qcut(data['model2_pred'], 4, labels=["1", "2", "3","4"])
data['scores_MAT_pred2'] = data['scores_MAT_pred2'].astype(int)
data['scores_MAT_pred3'] = pd.qcut(data['model3_pred'], 4, labels=["1", "2", "3","4"])
data['scores_MAT_pred3'] = data['scores_MAT_pred3'].astype(int)
data['scores_MAT_pred_C'] = pd.qcut(data['model3_pred_circum'], 4, labels=["1", "2", "3","4"])
data['scores_MAT_pred_C'] = data['scores_MAT_pred_C'].astype(int)
data['scores_MAT_pred_X'] = pd.qcut(data['model3_pred_effort'], 4, labels=["1", "2", "3","4"])
data['scores_MAT_pred_X'] = data['scores_MAT_pred_X'].astype(int)

# Transform predictions(continuous) to percentiles but percentiles 2 and 3 equal (between 25th and 75th percentile)

data['scores_MAT_pred1_t'] = data['scores_MAT_pred1'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))
data['scores_MAT_pred2_t'] = data['scores_MAT_pred2'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))
data['scores_MAT_pred3_t'] = data['scores_MAT_pred3'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))
data['scores_MAT_pred_C_t'] = data['scores_MAT_pred_C'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))
data['scores_MAT_pred_X_t'] = data['scores_MAT_pred_X'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))

## Results

We focus on **Equalized Odds** (Equality of opportunity).

To calculate Equalized Odds we first calculate recall or sensitivity:

$$TP / (TP + FN)$$

and then we calculate the ratio of recall among different groups to obtain Equalized Odds.

Recall is calculated for Low and High academic performance:
- **Low academic performance**: Below the median or 25th percentile
- **High academic performance**: Above the median or above 75th percentile (top 25 percent)

In [15]:
recall_dfs_25_75 = []
recall_dfs_25_75.extend(compute_recall(data, ["f12a"], top_level=5))
recall_dfs_25_75.extend(compute_recall(data, ["mother_education", "father_education", "mother_occupation", "father_occupation", "books"], top_level=4))
recall_dfs_25_75.extend(compute_recall(data, ["start_schooling_age"], top_level=1))
recall_dfs_25_75.extend(compute_recall(data, ["inmigrant_second_gen", "public_private", "capital_island", "a1", "ESCS_median", "ESCS_p25_p75", "d14"], top_level=1))

recall_dfs_between25_75 = []
recall_dfs_between25_75.extend(compute_recall_terciles(data, ["f12a"], top_level=5))
recall_dfs_between25_75.extend(compute_recall_terciles(data, ["mother_education", "father_education", "mother_occupation", "father_occupation", "books"], top_level=4))
recall_dfs_between25_75.extend(compute_recall_terciles(data, ["start_schooling_age"], top_level=1))
recall_dfs_between25_75.extend(compute_recall_terciles(data, ["inmigrant_second_gen", "public_private", "capital_island", "a1", "ESCS_median", "ESCS_p25_p75", "d14"], top_level=1))

recall_dfs_median = []
recall_dfs_median.extend(compute_recall_median(data, ["f12a"], top_level=5))
recall_dfs_median.extend(compute_recall_median(data, ["mother_education", "father_education", "mother_occupation", "father_occupation", "books"], top_level=4))
recall_dfs_median.extend(compute_recall_median(data, ["start_schooling_age"], top_level=1))
recall_dfs_median.extend(compute_recall_median(data, ["inmigrant_second_gen", "public_private", "capital_island", "a1", "ESCS_median", "ESCS_p25_p75", "d14"], top_level=1))

# Combine DataFrames
combined_df_25_75 = pd.concat(recall_dfs_25_75, ignore_index=True)
combined_df_between25_75 = pd.concat(recall_dfs_between25_75, ignore_index=True)
combined_df_median = pd.concat(recall_dfs_median, ignore_index=True)

In [16]:
# Pivot tables
pivot_combined_df_25_75 = combined_df_25_75.pivot_table(index=['Variable', 'Group', 'Percentile'], columns='Model', values='Recall').reset_index()
pivot_combined_df_25_75 = pivot_combined_df_25_75[['Variable', 'Group', 'Percentile', 'pred1', 'pred2', 'pred3', 'pred_C', 'pred_X']]
pivot_combined_df_25_75_sorted = pivot_combined_df_25_75.sort_values(by=['Percentile', 'Variable', 'Group'], ascending=[True, True, False])
pivot_combined_df_between25_75 = combined_df_between25_75.pivot_table(index=['Variable', 'Group', 'Tercile'], columns='Model', values='Recall').reset_index()
pivot_combined_df_between25_75 = pivot_combined_df_between25_75[['Variable', 'Group', 'Tercile', 'pred1_t', 'pred2_t', 'pred3_t', 'pred_C_t', 'pred_X_t']]
pivot_combined_df_between25_75_sorted = pivot_combined_df_between25_75.sort_values(by=['Tercile', 'Variable', 'Group'], ascending=[True, True, False])
pivot_combined_df_median = combined_df_median.pivot_table(index=['Variable', 'Group', 'Pair1', 'Pair2'], columns='Model', values='Recall').reset_index()
pivot_combined_df_median = pivot_combined_df_median[['Variable', 'Group', 'Pair1', 'Pair2', 'pred1', 'pred2', 'pred3', 'pred_C', 'pred_X']]
pivot_combined_df_median_sorted = pivot_combined_df_median.sort_values(by=['Pair1', 'Pair2', 'Variable', 'Group'], ascending=[True, True, True, False])

In [17]:
final_data_25_75 = []

for variable in pivot_combined_df_25_75_sorted['Variable'].unique():
    variable_df = pivot_combined_df_25_75_sorted[pivot_combined_df_25_75_sorted['Variable'] == variable]
    for percentile in variable_df['Percentile'].unique():
        top_row = variable_df[(variable_df['Group'] == 'top') & (variable_df['Percentile'] == percentile)]
        if not top_row.empty:
            top_row = top_row.iloc[0]
            temp_data = []
            for _, row in variable_df[variable_df['Percentile'] == percentile].iterrows():
                odds_row = {
                    'Variable': row['Variable'],
                    'Group': row['Group'],
                    'Percentile': row['Percentile'],
                    'pred1': row['pred1'],
                    'pred2': row['pred2'],
                    'pred3': row['pred3'],
                    'pred_C': row['pred_C'],
                    'pred_X': row['pred_X'],
                    'pred1_odds': calculate_odds(row['pred1'], top_row['pred1']),
                    'pred2_odds': calculate_odds(row['pred2'], top_row['pred2']),
                    'pred3_odds': calculate_odds(row['pred3'], top_row['pred3']),
                    'pred_C_odds': calculate_odds(row['pred_C'], top_row['pred_C']),
                    'pred_X_odds': calculate_odds(row['pred_X'], top_row['pred_X']),
                }
                temp_data.append(odds_row)
            final_data_25_75.extend(temp_data)

final_data_25_75_sorted = pd.DataFrame(final_data_25_75)

In [18]:
final_data_between25_75 = []

for variable in pivot_combined_df_between25_75_sorted['Variable'].unique():
    variable_df = pivot_combined_df_between25_75_sorted[pivot_combined_df_between25_75_sorted['Variable'] == variable]
    for tercile in variable_df['Tercile'].unique():
        top_row = variable_df[(variable_df['Group'] == 'top') & (variable_df['Tercile'] == tercile)]
        if not top_row.empty:
            top_row = top_row.iloc[0]
            temp_data = []
            for _, row in variable_df[variable_df['Tercile'] == tercile].iterrows():
                odds_row = {
                    'Variable': row['Variable'],
                    'Group': row['Group'],
                    'Tercile': row['Tercile'],
                    'pred1_t': row['pred1_t'],
                    'pred2_t': row['pred2_t'],
                    'pred3_t': row['pred3_t'],
                    'pred_C_t': row['pred_C_t'],
                    'pred_X_t': row['pred_X_t'],
                    'pred1_odds': calculate_odds(row['pred1_t'], top_row['pred1_t']),
                    'pred2_odds': calculate_odds(row['pred2_t'], top_row['pred2_t']),
                    'pred3_odds': calculate_odds(row['pred3_t'], top_row['pred3_t']),
                    'pred_C_odds': calculate_odds(row['pred_C_t'], top_row['pred_C_t']),
                    'pred_X_odds': calculate_odds(row['pred_X_t'], top_row['pred_X_t']),
                }
                temp_data.append(odds_row)
            final_data_between25_75.extend(temp_data)

final_data_between25_75_sorted = pd.DataFrame(final_data_between25_75)


In [19]:
final_data_median = []

for variable in pivot_combined_df_median_sorted['Variable'].unique():
    variable_df = pivot_combined_df_median_sorted[pivot_combined_df_median_sorted['Variable'] == variable]
    for pair in variable_df[['Pair1', 'Pair2']].drop_duplicates().values:
        pair1, pair2 = pair
        top_row = variable_df[(variable_df['Group'] == 'top') & (variable_df['Pair1'] == pair1) & (variable_df['Pair2'] == pair2)]
        if not top_row.empty:
            top_row = top_row.iloc[0]
            temp_data = []
            for _, row in variable_df[(variable_df['Pair1'] == pair1) & (variable_df['Pair2'] == pair2)].iterrows():
                odds_row = {
                    'Variable': row['Variable'],
                    'Group': row['Group'],
                    'Pair1': row['Pair1'],
                    'Pair2': row['Pair2'],
                    'pred1': row['pred1'],
                    'pred2': row['pred2'],
                    'pred3': row['pred3'],
                    'pred_C': row['pred_C'],
                    'pred_X': row['pred_X'],
                    'pred1_odds': calculate_odds(row['pred1'], top_row['pred1']),
                    'pred2_odds': calculate_odds(row['pred2'], top_row['pred2']),
                    'pred3_odds': calculate_odds(row['pred3'], top_row['pred3']),
                    'pred_C_odds': calculate_odds(row['pred_C'], top_row['pred_C']),
                    'pred_X_odds': calculate_odds(row['pred_X'], top_row['pred_X']),
                }
                temp_data.append(odds_row)
            final_data_median.extend(temp_data)

final_data_median_sorted = pd.DataFrame(final_data_median)


In [20]:
category_order = ['a1', 'mother_education', 'father_education', 'mother_occupation', 'father_occupation', 'books', 'd14', 'inmigrant_second_gen', 
                  'public_private', 'capital_island', 'start_schooling_age', 'f12a', 'ESCS_median', 'ESCS_p25_p75']

final_data_25_75_sorted['Variable'] = pd.Categorical(final_data_25_75_sorted['Variable'], categories=category_order, ordered=True)
final_data_25_75_sorted = final_data_25_75_sorted.sort_values(by='Variable')
final_data_25_75_sorted = final_data_25_75_sorted[['Variable', 'Group', 'Percentile', 'pred1', 'pred1_odds', 'pred2', 'pred2_odds', 'pred3', 'pred3_odds', 'pred_C', 'pred_C_odds', 'pred_X', 'pred_X_odds']]
final_data_25_75_sorted = final_data_25_75_sorted.sort_values(by=['Percentile', 'Variable', 'Group'], ascending=[True, True, False])
final_data_between25_75_sorted['Variable'] = pd.Categorical(final_data_between25_75_sorted['Variable'], categories=category_order, ordered=True)
final_data_between25_75_sorted = final_data_between25_75_sorted.sort_values(by='Variable')
final_data_between25_75_sorted = final_data_between25_75_sorted[['Variable', 'Group', 'Tercile', 'pred1_t', 'pred1_odds', 'pred2_t', 'pred2_odds', 'pred3_t', 'pred3_odds', 'pred_C_t', 'pred_C_odds', 'pred_X_t', 'pred_X_odds']]
final_data_between25_75_sorted = final_data_between25_75_sorted.sort_values(by=['Tercile', 'Variable', 'Group'], ascending=[True, True, False])
final_data_median_sorted['Variable'] = pd.Categorical(final_data_median_sorted['Variable'], categories=category_order, ordered=True)
final_data_median_sorted = final_data_median_sorted.sort_values(by='Variable')
final_data_median_sorted = final_data_median_sorted[['Variable', 'Group', 'Pair1', 'Pair2', 'pred1', 'pred1_odds', 'pred2', 'pred2_odds', 'pred3', 'pred3_odds', 'pred_C', 'pred_C_odds', 'pred_X', 'pred_X_odds']]
final_data_median_sorted = final_data_median_sorted.sort_values(by=['Pair1', 'Pair2', 'Variable', 'Group'], ascending=[True, True, True, False])

## Export to Excel

In [21]:
# Export to Excel
with pd.ExcelWriter(os.path.join('results', 'results.xlsx')) as writer:
    final_data_25_75_sorted.to_excel(writer, sheet_name='25_75', index=False, float_format='%.4f')
    final_data_median_sorted.to_excel(writer, sheet_name='Median', index=False, float_format='%.4f')
    final_data_between25_75_sorted.to_excel(writer, sheet_name='between25_75', index=False, float_format='%.4f')
    data.to_excel(writer, sheet_name='data', index=False, float_format='%.4f')

## IOP

Inequality of Opportunity is computed by applying an inequality index (Gini, MLD or simple variance) to the set of central moments (specifically, the mean $\mu$) for the $Y$'s conditional distributions with respect to a sensitive attribute's values. Mathematically:
$$IOP = I(w_{i} * \mu(Y|G_{i}), \ldots, w_{m} * \mu(Y|G_{m}))$$ 

$I$ is the inequality index while $G_{1} \ldots G_{m}$ are the $m$ different groups of individuals identified by the values a given senstive attribute can have. For example, if _gender_ is a sensitive attribute then the two resulting groups might be $G_{1} = male$ and $G_{2} = female$. In this situation, _IOP_ would be absent if $$I(w_{1} * \mu(Y|G_{1}), w_{2} * \mu(Y|G_{2})) = 0$$ or close to 0.

Finally, each central moment is weighted by the fraction of samples with a given value of the sensitive attribute. The weights sum up to 1. In the previous example, $w_{1}$ is the fraction of elements with $G = m$ and $w_{2}$ is the fraction of elements with $G = f$. 

In [21]:
data.columns

Index(['id_student_16_19', 'score_MAT', 'score_LEN', 'score_MAT3',
       'score_LEN3', 'a1', 'mother_education', 'father_education',
       'mother_occupation', 'father_occupation', 'inmigrant_second_gen',
       'start_schooling_age', 'books', 'f12a', 'public_private',
       'capital_island', 'd14', 'ESCS', 'id_school', 'scores_MATq',
       'scores_LENq', 'ESCS_median', 'ESCS_p25_p75', 'model1_pred',
       'model2_pred', 'Y_t_1_hat', 'ν_hat', 'model3_pred',
       'model3_pred_circum', 'mean_circu', 'model3_pred_effort',
       'scores_MAT_pred1', 'scores_MAT_pred2', 'scores_MAT_pred3',
       'scores_MAT_pred_C', 'scores_MAT_pred_X', 'scores_MAT_pred1_t',
       'scores_MAT_pred2_t', 'scores_MAT_pred3_t', 'scores_MAT_pred_C_t',
       'scores_MAT_pred_X_t'],
      dtype='object')

## Utility

In [22]:
def run_experiments(data, percentile, ineq_index):
    sensitive_attrs = ['a1', 'mother_education', 'father_education',
       'mother_occupation', 'father_occupation', 'inmigrant_second_gen',
       'start_schooling_age', 'books', 'f12a', 'public_private',
       'capital_island', 'd14', "ESCS_median", "ESCS_p25_p75"]

    model_preds = ["model1_pred", "model2_pred", "model3_pred", "model3_pred_circum", "model3_pred_effort"]

    model_pred_rename = {
        "model1_pred": "Model 1",
        "model2_pred": "Model 2",
        "model3_pred": "Model 3",
        "model3_pred_circum": "Circumstances",
        "model3_pred_effort": "Effort"
    }

    df = {}
    for mp in model_preds:
        col = []
        data["label"] = binarise_predictions(data[mp], percentile)
        for sa in sensitive_attrs:
            col.append(iop(data, sa, ineq_index=ineq_index))
        df[model_pred_rename[mp]] = col

    return pd.DataFrame(df, index=sensitive_attrs)

## Distributions

### Plot distributions

In [23]:
from matplotlib import pyplot as plt

# below 25th percentile

for pred in preds:
    data["label"] = binarise_predictions(data[pred], "below-25")
    for sa in sensitive_attrs:
        sensitive_vals = np.unique(data[sa].values[~np.isnan(data[sa].values)])
        fig, ax = plt.subplots(1, len(sensitive_vals), figsize=(20, 4))
        for idx, sv in enumerate(sensitive_vals):
            counts = data.loc[(data[sa] == sv)].label.value_counts()
            if len(counts) > 0:
                ax[idx].set_title(f"{model_pred_rename[pred]}, {sa} = {sv}")
                plot = counts.sort_values().plot(kind='barh', ax=ax[idx])
                ax[idx].set_xlabel("counts")
                ax[idx].set_ylabel("labels")
            else:
                print(f"No samples for {sa} = {sv}")
        fig.suptitle("Predictions below 25th percentile")
        fig.tight_layout()
        plt.show()
        plt.close()   

NameError: name 'preds' is not defined

## Below 25th percentile

In [24]:
df_below_25 = run_experiments(data=data, percentile="below-25", ineq_index="gini")
df_below_25

,Model 1,Model 2,Model 3,Circumstances,Effort
a1,0.012423,0.031248,0.027386,0.069062,0.003980
mother_education,0.373741,0.274751,0.264736,0.347400,0.499147
father_education,0.157341,0.180345,0.164012,0.381898,0.276662
mother_occupation,0.624813,0.618061,0.619149,0.604821,0.630546
father_occupation,0.609593,0.608956,0.607204,0.598686,0.610618
inmigrant_second_gen,0.494346,0.495191,0.494101,0.493823,0.494508
start_schooling_age,0.520788,0.491557,0.499742,0.447098,0.549294
books,0.407920,0.446337,0.453761,0.558274,0.367699
f12a,0.570050,0.553118,0.555135,0.543317,0.590276
public_private,0.431649,0.443198,0.449000,0.479572,0.397345


## Between 25th and 75th percentile

In [25]:
df_between_25_75 = run_experiments(data=data, percentile="between-25-75", ineq_index="gini")
df_between_25_75

,Model 1,Model 2,Model 3,Circumstances,Effort
a1,0.005187,0.005670,0.005670,0.006758,0.009530
mother_education,0.484025,0.492406,0.494423,0.488700,0.473725
father_education,0.261518,0.264349,0.268161,0.228309,0.251071
mother_occupation,0.636728,0.640020,0.640656,0.665600,0.632381
father_occupation,0.612205,0.618106,0.617775,0.636016,0.607198
inmigrant_second_gen,0.494546,0.494384,0.494586,0.493930,0.494647
start_schooling_age,0.542814,0.545352,0.544205,0.550030,0.540555
books,0.376009,0.392588,0.385189,0.480203,0.370630
f12a,0.583497,0.585618,0.586579,0.577092,0.580539
public_private,0.391809,0.393367,0.393247,0.414310,0.394678


## Above 75th percentile

In [26]:
df_above_75 = run_experiments(data=data, percentile="above-75", ineq_index="gini")
df_above_75

,Model 1,Model 2,Model 3,Circumstances,Effort
a1,0.028091,0.045923,0.042068,0.081562,0.003016
mother_education,0.574582,0.620994,0.619851,0.728787,0.492724
father_education,0.388726,0.497428,0.466972,0.676083,0.274500
mother_occupation,0.625452,0.625044,0.622531,0.594009,0.628681
father_occupation,0.599310,0.587734,0.590112,0.557446,0.608328
inmigrant_second_gen,0.495111,0.494589,0.495270,0.496793,0.494750
start_schooling_age,0.569744,0.588194,0.584185,0.610144,0.547564
books,0.332972,0.306212,0.311656,0.465067,0.366180
f12a,0.601265,0.612469,0.608837,0.649078,0.587460
public_private,0.366904,0.348476,0.340696,0.227034,0.398991


In [27]:
np_means = np.concatenate([
    df_below_25.mean(axis=0).to_numpy().reshape((1, df_below_25.shape[1])),
    df_above_75.mean(axis=0).to_numpy().reshape((1, df_above_75.shape[1])),
    df_between_25_75.mean(axis=0).to_numpy().reshape((1, df_between_25_75.shape[1]))], axis=0)

In [28]:
df_means = pd.DataFrame(np_means, 
                        index=["below_25", "above_75", "between_25_75"], 
                        columns=["Model 1", "Model 2", "Model 3", "Circumstances", "Effort"])
df_means

,Model 1,Model 2,Model 3,Circumstances,Effort
below_25,0.378705,0.393197,0.393307,0.450952,0.372639
above_75,0.397732,0.423386,0.415919,0.465793,0.368516
between_25_75,0.367508,0.371667,0.372826,0.380356,0.365642
